In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import math as m

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import os

In [2]:

lb = 1e-8
n_epoch = 2500
n_plot = 10
grad_max = 1e2
maxiters = 50000

lr_max = 5e-3
lr_min = 1e-5
lr_decay = 0.2
lr_decay_step = 500
w_decay = 1e-8

llb = lb
p_cutoff = -1.0
R = -1.0 / 8.314e-3

l_exp = np.arange(1,15)
n_exp = len(l_exp)

l_train = []
l_val = []
for i in np.arange(0,n_exp):
    j = l_exp[i]
    if not (j in [2, 6, 9, 12]):
        l_train.append(i+1)
    else:
        l_val.append(i+1)

# 初始参数设定

In [5]:

lb = 1e-8
n_epoch = 2500
n_plot = 10
grad_max = 1e2
maxiters = 50000

lr_max = 5e-3
lr_min = 1e-5
lr_decay = 0.2
lr_decay_step = 500
w_decay = 1e-8

llb = lb
p_cutoff = -1.0
R = -1.0 / 8.314e-3



nr=8
ns=4

npara = nr * (ns + 4) + 1

# 实验数据读取

## 目录定义

In [6]:
expr_name = "4s8r-01"
os.chdir("..")
print("当前工作目录：%s"%(str(os.getcwd())))

# 不要重复执行这段代码

当前工作目录：w:\CRNN-TF2.0\CRNN-TF2.0


In [10]:
fig_path = str("./results/" + expr_name + "/figs")
ckpt_path = str("./results/" + expr_name + "/checkpoint")
script_path = str("./scripts")

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path)


# 数据导入

In [17]:
def load_exp(filename):
    exp_data = np.loadtxt(filename)
    
    # 归一化质量数据
    exp_data[:, 2] = exp_data[:, 2] / max(exp_data[:,2])
    return tf.constant(exp_data, dtype=tf.float64)

# 共十四个实验，依次编号为1，2，...，14
l_exp = np.arange(1,15)
n_exp = len(l_exp)

# 训练集与验证集的数字编号列表，其中[2,6,9,12]为测试集
l_train = []
l_val = []
for i in np.arange(1,n_exp+1):
    j = l_exp[i-1]
    if not (j in [2, 6, 9, 12]):
        l_train.append(i)
    else:
        l_val.append(i)

l_exp_data = []
l_exp_info = np.zeros(shape=(len(l_exp), 3))



In [24]:
for i_exp, value in enumerate(l_exp):
    filename = str("./exp_data/expdata_no"+str(value)+".txt")
    exp_data = load_exp(filename)
    if value == 4:
        exp_data = exp_data[:60, :]
    elif value == 5:
        exp_data = exp_data[:58, :]
    elif value == 6:
        exp_data = exp_data[:60, :]
    elif value == 7:
        exp_data = exp_data[:71, :]
    l_exp_data.append(exp_data)

    # Initial temperature
    l_exp_info[i_exp-1, 0] = exp_data[0, 1]


l_exp_info[:, 1] = np.loadtxt("exp_data/beta.txt")
l_exp_info[:, 2] = np.log(np.loadtxt("exp_data/ocen.txt")+llb)

l_exp_info = tf.constant(l_exp_info, dtype=tf.float64)

# 初始化p矩阵

In [38]:
p = np.random.randn(npara) * 1.e-2
p[:nr] += 0.8 # w_b
# nr~nr*(ns+1) vij
p[nr*(ns+1):nr*(ns+2)] += 0.8 # w_out
p[nr*(ns+2):nr*(ns+4)] += 0.1 # w_b | w_Ea
p[-1] = 0.1 # slope

p = tf.Variable(p, dtype=tf.float64)

In [91]:
#测试
a = tf.Variable(np.arange(15), dtype=tf.float64)
b = tf.Variable([6,7,8,9,10], dtype=tf.float64)

c = tf.transpose(tf.reshape(a, shape=(3,5)))
print(c)
temp1 = tf.reshape(tf.reduce_sum(c[:4, :], 0),shape=(1,3))
c = tf.concat([c[:-2, :], temp1, c[-1:,:]],axis=0)
c

tf.Tensor(
[[ 0.  5. 10.]
 [ 1.  6. 11.]
 [ 2.  7. 12.]
 [ 3.  8. 13.]
 [ 4.  9. 14.]], shape=(5, 3), dtype=float64)


<tf.Tensor: shape=(5, 3), dtype=float64, numpy=
array([[ 0.,  5., 10.],
       [ 1.,  6., 11.],
       [ 2.,  7., 12.],
       [ 6., 26., 46.],
       [ 4.,  9., 14.]])>

# 定义CRNN方程

## 整理P矩阵

> 只能使用TensorFlow内部的张量操作，否则无法获得梯度。

In [105]:
@tf.function
def p2vec(p):
    slope = p[-1] * 1.e1
    
    w_b = p[:nr] * (slope * 10.0)
    w_b = tf.clip_by_value(w_b, clip_value_min=0., clip_value_max=50.)
    
    # julia 与 tf的reshape函数实现不同
    w_out = tf.reshape(p[nr:nr*(ns+1)],shape=(nr,ns))
    w_out = tf.transpose(w_out)

    temp1 = tf.clip_by_value(w_out[0:1, :], -3., 0.)
    temp2 = tf.clip_by_value(tf.abs(w_out[-1:, :]), 0., 3.)
    w_out = tf.concat([temp1, w_out[1:-1], temp2], axis=0)

    # 此段省略
    # julia原文：
    # if p_cutoff > 0.0
    #     w_out[findall(abs.(w_out) .< p_cutoff)] .= 0.0
    # end

    temp3 = -1*(tf.reduce_sum(w_out[:-2,:], 0) + w_out[-1:, :])
    temp3 = tf.reshape(temp3, shape=(1,nr))
    
    w_out = tf.concat([w_out[:-2,:], temp3, w_out[-1:,:]], axis=0)

    w_in_Ea = tf.abs(p[nr*(ns+1):nr*(ns+2)]) * slope * 100.
    w_in_Ea = tf.clip_by_value(w_in_Ea, 0., 300.)
    w_in_Ea = tf.reshape(w_in_Ea, shape=(1,8))

    w_in_b = tf.abs(p[nr*(ns+2):nr*(ns+3)])
    w_in_b = tf.reshape(w_in_b, shape=(1,8))

    w_in_ocen = tf.abs(p[nr*(ns+3):nr*(ns+4)])
    w_in_ocen = tf.clip_by_value(w_in_ocen, 0., 1.5)
    w_in_ocen = tf.reshape(w_in_ocen, shape=(1,8))

    # 此段省略
    # julia原文：
    # if p_cutoff > 0:
    #     w_in_ocean[abs(w_in_ocean)<p_cutoff] = 0.0
    
    w_in = tf.concat([
        tf.clip_by_value(w_out*(-1),0.,4.),
        w_in_Ea,
        w_in_b,
        w_in_ocen
    ], 0)

    return w_in, w_b, w_out

In [114]:
# 测试p
p_test = tf.constant(np.loadtxt("p.txt"))

## 定义CRNN结构

In [106]:
def getsampletemp(t, T0, beta):
    if beta<100:
        T = T0 + beta / 60 * t
    else:
        tc = tf.constant([999.0, 1059.0]) * 60.0
        Tc = tf.constant([beta, 370.0, 500.0]) + 273.0
        HR = 40.0/60.0
        if t <= tc[0]:
            T = Tc[0]
        elif t <= tc[1]:
            T = min(Tc[0] + HR * (t-tc[0]), Tc[1])
        else:
            
            T = min(Tc[1] + HR * (t-tc[1]), Tc[2])
    return T

In [116]:
@tf.function
def crnn(t, u):
    logX = tf.math.log(tf.reshape(tf.clip_by_value(u, lb, 10.), shape=(4,1)))
    T = getsampletemp(t, T0, beta)
    global p
    w_in, w_b, w_out = p2vec(p)
    w_in_X = tf.matmul(tf.transpose(w_in), tf.concat([logX, R/T, tf.log(T), ocen],0))
    du_dt = tf.matmul(w_out, tf.exp(tf.reshape(w_in_X, shape=(nr,1))+tf.reshape(w_b, shape=(nr,1))))
    return du_dt

u0 = tf.ones(shape=(ns,))

@tf.function
def pred_n_ode(p, i_exp, exp_data):
    global T0, beta, ocen
    T0 = l_exp_info[i_exp-1, 0]
    beta = l_exp_info[i_exp-1, 1]
    ocen = l_exp_info[i_exp-1, 2]
    ts = exp_data[:, 0]
    tspan = [ts[0],ts[-1]]

    solution = tf.function(lambda: tfp.math.ode.BDF().solve(
        crnn,
        tspan[0],#t_init
        u0,
        solution_times=ts
    ), autograph=True)()
    return solution.states


In [117]:
# pred_n_ode测试
u_test = pred_n_ode(p_test, 0, exp_data)

TypeError: in user code:

    File "<ipython-input-116-da124a9ce6cc>", line 22, in pred_n_ode  *
        solution = tf.function(lambda: tfp.math.ode.BDF().solve(
    File "<ipython-input-116-da124a9ce6cc>", line 4, in crnn  *
        T = getsampletemp(t, T0, beta)
    File "<ipython-input-106-2cd5d8a90ea0>", line 3, in getsampletemp  *
        T = T0 + beta / 60 * t

    TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type float64 of argument 'x'.


# 定义损失函数

In [107]:
def loss_n_ode(p, i_exp):
    exp_data = l_exp_data[i_exp-1]
    pred = pred_n_ode(p, i_exp, exp_data)
    
    masslist = tf.reduce_sum(tf.clip_by_value(pred[:-1,:], 0, 1e3),axis=0)
    gaslist = tf.clip_by_value(pred[-1:,:], 0, 1e3)

    loss = tf.keras.losses.MAE(masslist, exp_data[:len(masslist),2])

    if ocen < 1000.:
        loss += tf.keras.losses.MAE(gaslist, (1-exp_data[:len(masslist),2]))
    
    return loss

# 训练过程

In [ ]:
opt = ...
for epoch in range(n_epoch):
    print("Start epoch %d" %(epoch))
    
    for i_exp in (shuffled index of experiments):
        if i_exp in l_val:
            continue

        with tf.GradientTape() as tape:
            tape.watch(p)
            loss = loss_n_ode(p, i_exp)
        
        grad = tape.gradient(loss, p)
        opt.apply_gradients(zip([grad],[p]))
    
